In [1]:
import numpy as np
from datetime import datetime as dt, timedelta
from datetime import date
from enum import IntEnum
from dataclasses import dataclass
from dateutil.relativedelta import relativedelta
from typing import List, Dict, Any


class Frequency(IntEnum):
    ANNUAL = 1
    BIANNUAL = 2
    TRIANNUAL = 3
    QUARTERLY = 4
    MONTHLY = 12


@dataclass
class EquityShare:
    asset_id: int
    nace: str
    issuer: str
    buy_date: date
    dividend_yield: float
    frequency: Frequency
    market_price: float

    @property
    def dividend_amount(self) -> float:
        return self.dividend_yield # Probably needs to be removed

    def generate_dividend_dates(self, modelling_date: date, end_date: date ) -> date:
        """

        :type modelling_date: date
        """
        delta = relativedelta(months=(12 // self.frequency))
        this_date = self.buy_date - delta
        while this_date < end_date:  # Coupon payment dates
            this_date = this_date + delta
            if this_date < modelling_date: #Not interested in past payments
                continue
            if this_date <= end_date:
                yield this_date # ? What is the advantage of yield here?

#    def term_to_maturity(self, modelling_date: date)->int:
#        """
#
#        :type modelling_date: date
#        """
#        delta: timedelta = self.maturity_date - modelling_date
#        return delta.days

#    def gross_redemption_yield(self):
#        pass



In [19]:
test_share_1 # Test upload 1 share

EquityShare(asset_id=1, nace='A.1.2', issuer='Open Source Modelling', buy_date=datetime.date(2015, 12, 1), dividend_yield=0.03, frequency=<Frequency.QUARTERLY: 4>, market_price=12.6)

In [4]:
test_share_2 = EquityShare(asset_id= 2, nace= "A.3.1",
    issuer= "Test Issuer"
    ,buy_date= date(2020, 5, 2)
    ,dividend_yield= 0.04
    ,frequency= Frequency.ANNUAL
    ,market_price= 102.1
)

In [5]:
test_share_2 # Test upload 2 share

EquityShare(asset_id=2, nace='A.3.1', issuer='Test Issuer', buy_date=datetime.date(2020, 5, 2), dividend_yield=0.04, frequency=<Frequency.ANNUAL: 1>, market_price=102.1)

In [6]:
test_share_3 = EquityShare(asset_id= 3, nace= "B.12",
    issuer= "Test Issuer"
    ,buy_date= date(2020, 5, 2)
    ,dividend_yield= 0.012
    ,frequency= Frequency.TRIANNUAL
    ,market_price= 102
)

In [7]:
test_share_3 # Test upload 3 share

EquityShare(asset_id=3, nace='B.12', issuer='Test Issuer', buy_date=datetime.date(2020, 5, 2), dividend_yield=0.012, frequency=<Frequency.TRIANNUAL: 3>, market_price=102)

In [8]:
import pandas as pd
import csv
#from EquityClasses import EquityShare
from datetime import datetime


def GetEquityShare(filename) -> EquityShare:
    with open(filename, mode="r", encoding="utf-8-sig") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            equity_share = EquityShare(asset_id=int(row["Asset_ID"]),
                                 nace=row["NACE"],
                                 issuer=None,
                                 buy_date=datetime.strptime(row["Buy_Date"], "%d/%m/%Y").date(),
                                 dividend_yield=float(row["Dividend_Yield"]),
                                 frequency=int(row["Frequency"]),
                                 market_price=float(row["Market_Price"]))
            yield equity_share
            

In [9]:
test_equity_import = GetEquityShare("Input\Equity_Portfolio_test.csv")

In [10]:
class EquitySharePortfolio():
    def __init__(self, equity_share: dict[int,EquityShare] = None):
        """

        :type equity_share: dict[int,EquityShare]
        """
        self.equity_share = equity_share

    def IsEmpty(self)-> bool:
        if self.equity_share == None:
            return True
        if len(self.equity_share) == 0:
            return True
        return False

    def add(self,equity_share: EquityShare) :
        """

        :type equity_share: EquityShare
        """
        if self.equity_share == None:
            self.equity_share = {equity_share.asset_id: equity_share}
        else:
            self.equity_share.update({equity_share.asset_id: equity_share})



    def create_dividend_dates(self, modelling_date, end_date)->dict:
        """
                Create the vector of dates at which the dividends are paid out and the total amounts for
                all equity shares in the portfolio, for dates on or after the modelling date

                Parameters
                ----------
                self : EquitySharePortfolio class instance
                    The EquitySharePortfolio instance with populated initial portfolio.

                Returns
                -------
                EquityShare.coupondates
                    An array of datetimes, containing all the dates at which the coupons are paid out.

                """

        dividends: dict[date, float] = {}
        equity_share: EquityShare
        dividend_date: date
        for asset_id in self.equity_share:
            equity_share = self.equity_share[asset_id]
            dividend_amount = equity_share.dividend_amount
            for dividend_date in equity_share.generate_dividend_dates(modelling_date, end_date):
                if dividend_date in dividends:
                    dividends[dividend_date] = dividend_amount + dividends[dividend_date] # ? Why is here a plus? (you agregate coupon amounts if same date?)
                else:
                    dividends.update({dividend_date:dividend_amount})
        return dividends

    def create_terminal_cashflow(self, modelling_date: date, end_date: date) -> dict:
        """

        :rtype: dict
        """
        terminals: dict[date, float] = {}
        equity_share: EquityShare
        terminal_date: date

        for asset_id in self.equity_share:
            equity_share = self.equity_share[asset_id]
            #terminal_amount = equity_share.notional_amount
            terminal_amount = 1
            terminal_date = end_date
            if terminal_date in terminals:
                terminals[terminal_date] = terminal_amount + terminals[terminal_date]
            else:
                maturities.update({maturity_date:maturity_amount})
        return maturities


In [23]:
# Check equity construction
asset_id = 1
nace = "A.1.2"
issuer = "Open Source Modelling"
buy_date = date(2015, 12, 1)
dividend_yield = 0.03
frequency = Frequency.QUARTERLY
market_price = 12.6

test_share_1 = EquityShare(asset_id = asset_id, nace= nace,
    issuer= issuer
    ,buy_date= buy_date
    ,dividend_yield= dividend_yield
    ,frequency= frequency
    ,market_price= market_price
)

assert test_share_1.asset_id == asset_id
assert test_share_1.nace == nace
assert test_share_1.issuer == issuer
assert test_share_1.buy_date == buy_date
assert test_share_1.dividend_yield == dividend_yield







In [11]:
# Test is Portfolio constructor
a = EquitySharePortfolio()


In [12]:
#test_Not_IsEmpty
b = EquitySharePortfolio({test_share_1.asset_id: test_share_1}) 

In [13]:
# test_add_to_empty_portfolio
equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)

# isEmpty?
equity_share_portfolio.IsEmpty()

# Only one equity in ptf
len(equity_share_portfolio.equity_share)

# Is equity identified by asset id equal to the input equity object
equity_share_portfolio.equity_share[test_share_1.asset_id] == test_share_1

test_share_1.asset_id in equity_share_portfolio.equity_share


True

In [14]:
# test_add_to_non_empty_portfolio

equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)
equity_share_portfolio.add(test_share_2)
equity_share_portfolio.add(test_share_3)

len(equity_share_portfolio.equity_share)

test_share_3.asset_id in equity_share_portfolio.equity_share

True

In [16]:
#test_create_dividend_dates_single_bond(test_share_1):
equity_share_portfolio = EquitySharePortfolio()
equity_share_portfolio.add(test_share_1)
modelling_date = date(2023, 6, 1)
end_date = date(2023+50, 6, 1)
dividend_dates = equity_share_portfolio.create_dividend_dates(modelling_date, end_date)

assert date(2023, 6, 1) in dividend_dates
assert date(2023, 9, 1) in dividend_dates
assert date(2023, 12, 1) in dividend_dates
#assert test_share_1.maturity_date in dividend_dates  # TODO: Check that a final coupon should be paid at maturity
assert dividend_dates[date(2023, 6, 1)] == test_share_1.dividend_amount


True